In [1]:
%matplotlib inline

# Imports

In [2]:
import joblib
import pandas as pd
import glob

import numpy as np
from matplotlib import pyplot as plt

# Plotting parameters

In [3]:
import matplotlib
###Default for jupyter display
matplotlib.rcParams['figure.dpi']= 150

matplotlib.rcParams['xtick.labelsize'] = 7
matplotlib.rcParams['ytick.labelsize'] = 7
matplotlib.rcParams['axes.labelsize'] = 7
matplotlib.rcParams['axes.titlesize'] = 7

matplotlib.rcParams['axes.grid'] = True
matplotlib.rcParams['grid.color'] = '0.8'
matplotlib.rcParams['grid.linewidth'] = '0.5'

matplotlib.rcParams['axes.edgecolor'] = '0.25'
matplotlib.rcParams['xtick.color'] = '0'
matplotlib.rcParams['ytick.color'] = '0'

matplotlib.rcParams['xtick.major.width'] = 1
matplotlib.rcParams['ytick.major.width'] = 1
matplotlib.rcParams['ytick.major.size'] = 5
matplotlib.rcParams['xtick.major.size'] = 5
matplotlib.rcParams['axes.spines.right'] = True
matplotlib.rcParams['axes.spines.left'] = True
matplotlib.rcParams['axes.spines.top'] = True
matplotlib.rcParams['axes.spines.bottom'] = True

matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = 'Helvetica'
matplotlib.rcParams['font.weight']='normal'
matplotlib.rcParams['axes.axisbelow'] = True
# matplotlib.rcParams['text.usetex'] = True

matplotlib.rcParams['legend.fontsize'] = 7
matplotlib.rcParams['legend.handlelength'] = 1
matplotlib.rcParams['legend.handleheight'] = 1
matplotlib.rcParams['legend.handletextpad'] = 0.4



prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

# Read in / process data

In [4]:
###Load classifier model
clf = joblib.load('../Data/classifier_data/rf_version_updating.joblib')

###Load datasets
train_df = pd.read_csv('../Data/classifier_data/train_df.csv', index_col=0)
print(train_df.shape)
test_df = pd.read_csv('../Data/classifier_data/test_df.csv', index_col=0)
print(test_df.shape)

(634, 229)
(423, 229)


**Integrate my predictions into the training/testing dataframes**

In [5]:
train_df['my_predictions'] = clf.predict(train_df[train_df.columns[23:]])
test_df['my_predictions'] = clf.predict(test_df[test_df.columns[23:]])

In [6]:
train_df['my_predictions'] = train_df['my_predictions'].replace(1, 'yes')
train_df['my_predictions'] = train_df['my_predictions'].replace(0, 'no')

test_df['my_predictions'] = test_df['my_predictions'].replace(1, 'yes')
test_df['my_predictions'] = test_df['my_predictions'].replace(0, 'no')

**Integrate PHACTS predictions into dataframe**

In [7]:
for df in [train_df, test_df]:
    listy = []
    for index in df.index[:]:
        name = df.loc[index]['Identifier_AJH']
        phacts_out = '../Data/phage_data_nmicro2017/PHACTS_results/{}_prodigal.out'.format(name)
        name2 = phacts_out.split('/')[-1].split('_prodigal.out')[0]
        assert name == name2
        phacts_df = pd.read_csv(phacts_out, sep='\t', skiprows=3, header=None)
        assert phacts_df.iloc[0][1] >= phacts_df.iloc[1][1]
        if phacts_df.iloc[0][1] == phacts_df.iloc[1][1]:
            listy.append('-')
            continue
        if phacts_df.iloc[0][0] == 'Temperate':
            listy.append('yes')
        else:
            listy.append('no')
    df['Temperate_PHACTS'] = listy

# Assess accuracies

In [8]:
###Training set
my_train_errors = train_df[train_df['Temperate (empirical)'] != 
                           train_df['my_predictions']].shape[0]

mavrich_train_errors = train_df[train_df['Temperate (empirical)'] != 
                           train_df['Temperate (bioinformatically predicted)']].shape[0]

phacts_train_errors = train_df[train_df['Temperate (empirical)'] != 
                           train_df['Temperate_PHACTS']].shape[0]

print('Out of a total {}'.format(train_df.shape[0]))
print('My method had {}'.format(my_train_errors))
print('Other method had {}'.format(mavrich_train_errors))
print('PHACTS method had {}'.format(phacts_train_errors))

my_train_errors = my_train_errors / train_df.shape[0] * 100
mavrich_train_errors = mavrich_train_errors / train_df.shape[0] * 100
phacts_train_errors = phacts_train_errors / train_df.shape[0] * 100

Out of a total 634
My method had 1
Other method had 24
PHACTS method had 120


In [9]:
###Test set
my_test_errors = test_df[test_df['Temperate (empirical)'] != 
                           test_df['my_predictions']].shape[0]

mavrich_test_errors = test_df[test_df['Temperate (empirical)'] != 
                           test_df['Temperate (bioinformatically predicted)']].shape[0]

phacts_test_errors = test_df[test_df['Temperate (empirical)'] != 
                           test_df['Temperate_PHACTS']].shape[0]

print('Out of a total {}'.format(test_df.shape[0]))
print('My method had {}'.format(my_test_errors))
print('Other method had {}'.format(mavrich_test_errors))
print('PHACTS method had {}'.format(phacts_test_errors))

my_test_errors = my_test_errors / test_df.shape[0] * 100
mavrich_test_errors = mavrich_test_errors / test_df.shape[0] * 100
phacts_test_errors = phacts_test_errors / test_df.shape[0] * 100

Out of a total 423
My method had 7
Other method had 19
PHACTS method had 89


In [10]:
1-(19/423)
# 1-(1/634)

0.9550827423167849

# Further split the test set into easy/difficult sets

**According to some pre-calculated clusters of data, this splits the test set up specifically according to whether or not a related sequence (from the same cluster) was / was not included in the training set. The goal is to assess accuracy on an un-polluted (as much as possible) test set**

See `cluster_seqs.ipynb` for details

In [ ]:
import json
with open('../Data/fastANI_output/clusters.json', 'r') as infile:
    clusters = json.load(infile)

In [ ]:
temp_names = list(train_df['Identifier_AJH'])
independent_set = []
related_set = []
for cluster in clusters:
    hits = []
    for member in cluster:
        if member in temp_names:
            hits.append(member)
    if len(hits) == 0:
        independent_set.extend(cluster)
    else:
        related_set.extend(cluster)
###Get the challenging/easy sets        
independent_df = test_df[test_df['Identifier_AJH'].isin(independent_set)]
print(independent_df.shape)
related_df = test_df[test_df['Identifier_AJH'].isin(related_set)]
print(related_df.shape)

In [ ]:
my_independent_errors = independent_df[independent_df['Temperate (empirical)'] != 
                           independent_df['my_predictions']].shape[0] /\
                            independent_df.shape[0] * 100.

mavrich_independent_errors = independent_df[independent_df['Temperate (empirical)'] != 
                           independent_df['Temperate (bioinformatically predicted)']].shape[0] /\
                            independent_df.shape[0] * 100.

phacts_independent_errors = independent_df[independent_df['Temperate (empirical)'] != 
                           independent_df['Temperate_PHACTS']].shape[0] /\
                            independent_df.shape[0] * 100.



my_related_errors = related_df[related_df['Temperate (empirical)'] != 
                           related_df['my_predictions']].shape[0] /\
                            related_df.shape[0] * 100.

mavrich_related_errors = related_df[related_df['Temperate (empirical)'] != 
                           related_df['Temperate (bioinformatically predicted)']].shape[0] /\
                            related_df.shape[0] * 100.

phacts_related_errors = related_df[related_df['Temperate (empirical)'] != 
                           related_df['Temperate_PHACTS']].shape[0] /\
                            related_df.shape[0] * 100.

In [ ]:
100-phacts_independent_errors

# Make a final plot of the error rates

In [ ]:
N = 4
my_errs = [my_train_errors, my_test_errors, my_related_errors, my_independent_errors][::-1]
mavrich_errs = [mavrich_train_errors, mavrich_test_errors, mavrich_related_errors, mavrich_independent_errors][::-1]
phacts_errs = [phacts_train_errors, phacts_test_errors, phacts_related_errors, phacts_independent_errors][::-1]

fig, ax = plt.subplots(figsize=(2.7,1.7))
ind = np.arange(N)    # the x locations for the groups
width = 0.25         # the width of the bars
p1 = ax.barh(ind, phacts_errs, width)
p2 = ax.barh(ind+width, mavrich_errs, width)
p3 = ax.barh(ind+width+width, my_errs, width)


ax.set_yticks(0.25 + np.arange(N))
ax.set_yticklabels(['Training set\n(n={})'.format(train_df.shape[0]),\
                    'Testing set\n(n={})'.format(test_df.shape[0]),\
                    'Testing set, rel.\n(n={})'.format(related_df.shape[0]),\
                    'Testing set, ind.\n(n={})'.format(independent_df.shape[0])][::-1],\
                   ha='center', )
ax.yaxis.get_majorticklabels()[0].set_x(-0.17)
ax.yaxis.get_majorticklabels()[1].set_x(-0.17)
ax.yaxis.get_majorticklabels()[2].set_x(-0.17)
ax.yaxis.get_majorticklabels()[3].set_x(-0.17)



ax.legend((p3[0], p2[0], p1[0]), ('BACPHLIP', 'Mavrich', 'PHACTS'),\
          bbox_to_anchor=(0., 1.02, 1., .102), loc=3, mode='expand', ncol=3, borderaxespad=0.)

ax.set_xlabel('% Incorrect predictions');
plt.savefig('../Manuscript/figure.png', dpi=300, bbox_inches='tight')

# Confusion matrices and other assorted metrics

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, balanced_accuracy_score, accuracy_score,matthews_corrcoef

In [ ]:
print(train_df.columns[:22])

In [ ]:
400/423

In [ ]:
temp_df = test_df
print(temp_df['Temperate (empirical)'].value_counts())
print()
print('BACPHLIP')
print(confusion_matrix(temp_df['Temperate (empirical)'], temp_df['my_predictions']))
print()
print('Mavrich')
print(confusion_matrix(temp_df['Temperate (empirical)'], temp_df['Temperate (bioinformatically predicted)']))
print()
print('PHACTS')
print(confusion_matrix(temp_df['Temperate (empirical)'], temp_df['Temperate_PHACTS']))

In [ ]:
temp_df = test_df
print(temp_df['Temperate (empirical)'].value_counts())
print()
print('BACPHLIP')
print(accuracy_score(temp_df['Temperate (empirical)'].map(dict(yes=1, no=0)),\
               temp_df['my_predictions'].map(dict(yes=1, no=0))))
print()
print('Mavrich')
print(accuracy_score(temp_df['Temperate (empirical)'].map(dict(yes=1, no=0)),\
               temp_df['Temperate (bioinformatically predicted)'].map(dict(yes=1, no=0))))
print()
print('PHACTS')
print(accuracy_score(temp_df['Temperate (empirical)'].map(dict(yes=1, no=0)),\
               temp_df['Temperate_PHACTS'].map(dict(yes=1, no=0))))

In [ ]:
temp_df = test_df
print(temp_df['Temperate (empirical)'].value_counts())
print()
print('BACPHLIP')
print(balanced_accuracy_score(temp_df['Temperate (empirical)'].map(dict(yes=1, no=0)),\
               temp_df['my_predictions'].map(dict(yes=1, no=0)), adjusted=True))
print()
print('Mavrich')
print(balanced_accuracy_score(temp_df['Temperate (empirical)'].map(dict(yes=1, no=0)),\
               temp_df['Temperate (bioinformatically predicted)'].map(dict(yes=1, no=0)), adjusted=True))
print()
print('PHACTS')
print(balanced_accuracy_score(temp_df['Temperate (empirical)'].map(dict(yes=1, no=0)),\
               temp_df['Temperate_PHACTS'].map(dict(yes=1, no=0)), adjusted=True))

In [ ]:
temp_df = test_df
print(temp_df['Temperate (empirical)'].value_counts())
print()
print('BACPHLIP')
print(f1_score(temp_df['Temperate (empirical)'].map(dict(yes=1, no=0)),\
               temp_df['my_predictions'].map(dict(yes=1, no=0))))

print()
print('Mavrich')
print(f1_score(temp_df['Temperate (empirical)'].map(dict(yes=1, no=0)),\
               temp_df['Temperate (bioinformatically predicted)'].map(dict(yes=1, no=0))))

print()
print('PHACTS')
print(f1_score(temp_df['Temperate (empirical)'].map(dict(yes=1, no=0)),\
               temp_df['Temperate_PHACTS'].map(dict(yes=1, no=0))))

In [ ]:
temp_df = test_df
print(temp_df['Temperate (empirical)'].value_counts())
print()
print('BACPHLIP')
print(matthews_corrcoef(temp_df['Temperate (empirical)'].map(dict(yes=1, no=0)),\
               temp_df['my_predictions'].map(dict(yes=1, no=0))))

print()
print('Mavrich')
print(matthews_corrcoef(temp_df['Temperate (empirical)'].map(dict(yes=1, no=0)),\
               temp_df['Temperate (bioinformatically predicted)'].map(dict(yes=1, no=0))))

print()
print('PHACTS')
print(matthews_corrcoef(temp_df['Temperate (empirical)'].map(dict(yes=1, no=0)),\
               temp_df['Temperate_PHACTS'].map(dict(yes=1, no=0))))

# Looking for differences related to genome size

In [ ]:
from scipy import stats

In [ ]:
temp_df = test_df[test_df['my_predictions'] != test_df['Temperate (empirical)']]
print(temp_df.shape)
incorrect_sizes = temp_df['Genome size']

temp_df = test_df[test_df['my_predictions'] == test_df['Temperate (empirical)']]
print(temp_df.shape)
correct_sizes = temp_df['Genome size']

In [ ]:
print(np.mean(incorrect_sizes), np.mean(correct_sizes))
print(np.median(incorrect_sizes), np.median(correct_sizes))
print(stats.ranksums(incorrect_sizes, correct_sizes))

# High confidence predictions

In [30]:
preds =  clf.predict_proba(test_df[test_df.columns[23:-1]])
preds = np.array(list(zip(*preds))[1])

In [31]:
###Test set
print('Errors on high confidence predictions:')
print(test_df[(test_df['Temperate (empirical)'] != 
                           test_df['my_predictions']) &\
                         ((preds>=0.95) | (preds<=0.05))].shape[0])
print('Total high confidence predictions:')
print(test_df[((preds>=0.95) | (preds<=0.05))].shape[0])

Errors on high confidence predictions:
1
Total high confidence predictions:
333


In [32]:
test_df[((preds>=0.95) | (preds<=0.05))]['my_predictions'].value_counts()

yes    201
no     132
Name: my_predictions, dtype: int64